In [32]:
import csv
import time
import requests
from bs4 import BeautifulSoup

# URL de la página web a scrapear
url = "https://www.portfoliopersonal.com/Cotizaciones/Bonos"

# Ruta del archivo CSV
ruta_csv = "/Users/mrz/Documents/jupyter/anker/precio.csv"  # Reemplaza "/path/to" con la ruta deseada para guardar el archivo CSV

while True:
    # Realizar la solicitud HTTP GET a la página
    response = requests.get(url)

    # Comprobar el estado de la respuesta
    if response.status_code == 200:
        # Crear un objeto BeautifulSoup con el contenido HTML de la página
        soup = BeautifulSoup(response.content, "html.parser")

        # Encontrar la tabla de cotizaciones
        tabla_cotizaciones = soup.find("table", class_="table")

        if tabla_cotizaciones is not None:
            # Obtener las filas de la tabla
            filas = tabla_cotizaciones.find_all("tr")

            # Crear la lista de datos
            datos = []
            for fila in filas:
                celdas = fila.find_all("td")
                datos_fila = [celda.text.strip() for celda in celdas]
                datos.append(datos_fila)

            # Guardar los datos en un archivo CSV
            with open(ruta_csv, "w", newline="") as archivo_csv:
                escritor_csv = csv.writer(archivo_csv)
                for fila in datos:
                    escritor_csv.writerow(fila)

            print("Archivo CSV guardado correctamente a las", time.strftime("%H:%M:%S"), "horas.")
        else:
            print("No se encontró la tabla de cotizaciones en la página.")
    else:
        print("Error al obtener la página:", response.status_code)

    # Esperar 20 minutos antes de la siguiente ejecución
    time.sleep(20 * 60)  # 20 minutos en segundos (20 * 60)


FileNotFoundError: [Errno 2] No such file or directory: '/path/to/precio.csv'